<a href="https://colab.research.google.com/github/laysfreitas/Covid_Dados/blob/main/LSTM_Covid_Data_Obitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importações**

In [1]:
import requests
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# **Carregando os dados**

In [2]:
!gdown 'https://drive.google.com/uc?id=1Zi5iwuWt0fsH0uVOiyWZHctUX0pV_De7'

Downloading...
From: https://drive.google.com/uc?id=1Zi5iwuWt0fsH0uVOiyWZHctUX0pV_De7
To: /content/R_GO.csv
100% 861/861 [00:00<00:00, 6.00MB/s]


In [3]:
DF_Vacinacao = pd.read_csv('V_GO.csv')

In [4]:
DF = pd.read_csv('/content/R_GO.csv',sep=',')

In [5]:
DF.head()

,Unnamed: 0,saidaConfirmadaObitos,casosNovos
0,Janeiro 2020,0.0,0.0
1,Fevereiro 2020,0.0,0.0
2,Março 2020,2.0,130.0
3,Abril 2020,56.0,1432.0
4,Maio 2020,190.0,5842.0


In [6]:
DF_Vacinacao.head()

,Unnamed: 0,1ª Dose,2ª Dose,Dose Adicional,Reforço,2ª Dose Revacinação,Dose,1ª Dose Revacinação,2º Reforço,Revacinação,3ª Dose Revacinação,3º Reforço,4ª Dose Revacinação,Dose Inicial,3ª Dose,5ª Dose
0,Janeiro-2021,31832.0,57.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Fevereiro-2021,43821.0,23495.0,5.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Março-2021,141303.0,29779.0,11.0,11.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abril-2021,112493.0,129479.0,4.0,4.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Maio-2021,183377.0,43527.0,0.0,5.0,0.0,4.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Preparando os dados**

In [7]:
DF_Vacinacao = DF_Vacinacao.iloc[:,[0,1,2,4]]

In [8]:
DF_Vacinacao['Doses'] = DF_Vacinacao['1ª Dose'] + DF_Vacinacao['2ª Dose'] + DF_Vacinacao['Reforço']

In [9]:
DF_Vacinacao['Unnamed: 0'] = DF_Vacinacao['Unnamed: 0'].str.replace("-"," ")

In [10]:
DF_Vacinacao_2020 = pd.DataFrame({'Unnamed: 0': ['Janeiro 2020', 'Fevereiro 2020', 'Março 2020', 'Abril 2020','Maio 2020','Junho 2020','Julho 2020','Agosto 2020','Setembro 2020','Outubro 2020','Novembro 2020','Dezembro 2020'],
                                     '1ª Dose': [0,0,0,0,0,0,0,0,0,0,0,0],
                                     '2ª Dose': [0,0,0,0,0,0,0,0,0,0,0,0],
                                     'Reforço': [0,0,0,0,0,0,0,0,0,0,0,0],
                                     'Doses': [0,0,0,0,0,0,0,0,0,0,0,0]})

In [11]:
DF_Vac = pd.concat([DF_Vacinacao_2020, DF_Vacinacao])

In [12]:
DF_TOTAL = pd.merge(DF, DF_Vac, how = 'inner', on = 'Unnamed: 0')
DF_TOTAL.rename(columns={'Unnamed: 0': 'Data'}, inplace = True)
DF_TOTAL = DF_TOTAL.iloc[:,[1,2,6]]

In [13]:
DF_TOTAL

,saidaConfirmadaObitos,casosNovos,Doses
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,2.0,130.0,0.0
3,56.0,1432.0,0.0
4,190.0,5842.0,0.0
5,700.0,41358.0,0.0
6,2364.0,87120.0,0.0
7,2876.0,129130.0,0.0
8,3156.0,153810.0,0.0
9,2162.0,92072.0,0.0


# **Modelo LSTM**

Esse código cria um modelo Sequential de Rede Neural Artificial com 3 camadas LSTM, com 50 unidades cada, seguidas por duas camadas de Dropout (com taxa de descarte de 0,2), e finalmente, uma camada densa com uma unidade. A função de perda utilizada é a "mean_squared_error" e o otimizador escolhido é o "adam". A forma da entrada da camada LSTM é definida como uma matriz tridimensional de formato (xin.shape[1], 1), onde xin.shape[1] representa o comprimento da janela de tempo.



*   Mean Squared Error (MSE): A função MSE calcula a média dos erros ao quadrado entre as previsões e os valores reais. É amplamente utilizada em problemas de regressão.
*   Mean Squared Logarithmic Error (MSLE): A MSLE é semelhante à MSE, mas aplica a logaritmo aos erros antes de calcular a média. Isso é útil para problemas onde as diferenças entre as previsões e os valores reais são grandes.





* Adam: um otimizador baseado em gradiente que utiliza técnicas de adaptação de taxa de aprendizado
*  RMSprop: um otimizador baseado em gradiente que se concentra na descida de gradiente em direção às direções de descida de gradiente mais rápidas



Verificar o score R². Se o score R² for muito alto nos dados de treinamento, mas muito baixo nos dados de validação, é provável que o modelo esteja sofrendo de overfitting.

**Importar bibliotecas**

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

**Dados de teste e treino**

In [15]:
def reshape_data(data, n_timesteps):
  n_samples, n_features = data.shape
  return data.reshape((n_samples - n_timesteps + 1, n_timesteps, n_features))

In [16]:
features = DF_TOTAL[['casosNovos', 'Doses']]
target = DF_TOTAL['saidaConfirmadaObitos']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# n_timesteps = 10  # Ajuste o valor de n_timesteps de acordo com o tamanho do histórico que você deseja usar
# X_train = reshape_data(X_train, n_timesteps)
# X_test = reshape_data(X_test, n_timesteps)
# n_features = reshape_data(features,n_timesteps)

# Reshape data without using n_timesteps
aux1 = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
aux2 = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

n_features = aux1.shape[1]
n_timesteps = aux2.shape[0]



**Construir o modelo LSTM**

In [17]:
model = Sequential()
# Encoder LSTM layers (accepts the input sequence)
model.add(LSTM(units=128, return_sequences=True, input_shape=(n_timesteps, n_features)))
model.add(LSTM(units=100, return_sequences=True))

# Decoder LSTM layers (produce the output sequence)
model.add(LSTM(units=50, return_sequences=True))  # Note the return_sequences=True here
# Output layer (dense layer to predict the output sequence)
model.add(Dense(units=1))

**Compilar modelo**

In [18]:
model.compile(optimizer='adam', loss='mse')

**Fit do modelo**

In [19]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


ValueError: ignored

**Fazer previsões**

In [ ]:
predictions = model.predict(X_test)